# SelectorGroupChat


SelectorGroupChat は、エージェントが順番に他のエージェント全員にメッセージをブロードキャストするチームを実装しています。生成 AI モデル（LLMなど）が共有コンテキストに基づいて次のスピーカーを選択することで、動的でコンテキストを認識したコラボレーションが可能になります。

- Selector による中央集権的なエージェント割り振り
- 遷移先の決定: LLM+ルール
- コンテキスト: 共有

https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/selector-group-chat.html

In [ ]:
#!pip install -U "autogen-agentchat"
#!pip install "autogen-ext[mcp,openai,azure]"

In [ ]:
import logging  
from typing import Any, List  
import os  
from dotenv import load_dotenv  

from autogen_agentchat.agents import AssistantAgent  
from autogen_agentchat.teams import SelectorGroupChat  # keeps implementation simple & familiar  
from autogen_agentchat.conditions import TextMessageTermination, MaxMessageTermination, TextMentionTermination, TimeoutTermination

from autogen_core import CancellationToken  
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient  
from autogen_ext.tools.mcp import SseServerParams, mcp_server_tools  
import warnings
warnings.simplefilter('ignore')

load_dotenv()

## OpenTelemetry によるトレーサーのセット
マルチエージェントのデバッグには OpenTelemetry によるトレーサーを利用すると便利。`OpenAIInstrumentor` を使用して OpenAI コールをキャプチャできます。ここではトレース UI として [Jaeger](https://www.jaegertracing.io/download/) を使用しています。

In [ ]:
#!pip install opentelemetry-api opentelemetry-sdk opentelemetry-exporter-otlp opentelemetry-instrumentation-openai

In [ ]:
from opentelemetry import trace
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor
from opentelemetry.exporter.otlp.proto.grpc.trace_exporter import OTLPSpanExporter
from opentelemetry.instrumentation.openai import OpenAIInstrumentor

service_name = "autogen"

# OTLPエクスポーターの設定 (gRPC経由で送信)
otlp_exporter = OTLPSpanExporter(
    endpoint="http://localhost:4317",  # JaegerのgRPCエンドポイント
)
tracer_provider = TracerProvider(resource=Resource({"service.name": service_name}))
    
# トレーサープロバイダーの設定
trace.set_tracer_provider(tracer_provider)

# バッチスパンプロセッサーを設定
span_processor = BatchSpanProcessor(otlp_exporter)
tracer_provider.add_span_processor(span_processor)

# トレーサーを取得
tracer = tracer_provider.get_tracer(service_name)

OpenAIInstrumentor().instrument()

In [ ]:
mcp_server_uri = os.getenv("MCP_SERVER_URI")
azure_openai_key = os.getenv("AZURE_OPENAI_KEY")
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_model = os.getenv("AZURE_OPENAI_MODEL")
azure_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")
openai_model_name = os.getenv("OPENAI_MODEL_NAME")

## Tools の定義(MCP over SSE)

In [ ]:

server_params = SseServerParams(  
    url=mcp_server_uri,  
    headers={"Content-Type": "application/json"},  
    timeout=30  
)  

# Fetch tools (async)  
tools = await mcp_server_tools(server_params)  
print(f"Number of Tools: {len(tools)}")
# Set up the OpenAI/Azure model client  
model_client = AzureOpenAIChatCompletionClient(  
    api_key=azure_openai_key,  
    azure_endpoint=azure_openai_endpoint,  
    api_version=api_version,  
    azure_deployment=azure_deployment,  
    model=openai_model_name,  
)  

## エージェント定義
### Handsoff パラメーター
Swarm の場合、AssistantAgent に handoffs 引数を設定して、どのエージェントにハンドオフ（委譲）できるかを指定することができます。Selector のような遷移先を決めるマネージャーが存在しないため、遷移先の決定は各エージェントのプロンプトに依存します

Handoff pydantic model を使用して、メッセージの内容とハンドオフの動作をカスタマイズすることもできます。

In [ ]:

# 3. -----------------  Agent Definitions -----------------  
analysis_planning_agent = AssistantAgent(  
    name="AnalysisPlanningAgent",  
    model_client=model_client,  
    description="タスクを計画するエージェント。新しいタスクが与えられたときに最初に起動するエージェントであるべきである。",
    system_message=(  
"""
あなたは「分析 & 計画エージェント（Analysis & Planning Agent）」です – 全体のオーケストレーターとして機能します。

あなたの役割:
1) 顧客からの抽象的なリクエストを解析すること。
2) リクエストを明確なサブタスクに分解し、それぞれを専門エージェント
   （crm_billing、product_promotions）に割り当てること。
3) 各専門エージェントからの出力を統合し、1つの包括的で一貫性のある
   顧客向けの回答としてまとめること。
4) 満足のいく結果が得られたら、以下のプレフィックスを付けて顧客に最終回答すること
   TERMINATE

まだ情報が不足している場合は、専門エージェントとの対話を継続してください。
情報が揃っていれば調査結果を要約し、文の最後に TERMINATE を含めること!

"""
)
)  

crm_billing_agent = AssistantAgent(  
    name="CRMBillingAgent",  
    model_client=model_client,  
    description="CRM & 請求エージェントのエージェント。CRM／請求システムを照会する",
    tools=tools,  
    system_message=(
"""
あなたは「CRM & 請求エージェント（CRM & Billing Agent）」です。

- アカウント、サブスクリプション、請求書、支払い情報などを取得するために、
  構造化されたCRM／請求システムを照会します。
- 請求ポリシー、支払い処理、返金ルールなどに関する *ナレッジベース* 記事を確認し、
  回答が正確かつポリシーに準拠していることを保証します。
- 簡潔で構造化された情報を返答し、検出されたポリシー上の懸念事項があれば
  フラグを立てます。
"""
    ),  
)  

product_promotions_agent = AssistantAgent(  
    name="ProductPromotionsAgent",  
    model_client=model_client,  
    description="製品 & プロモーションエージェント。プロモーションのオファー、製品の在庫状況、適格条件、割引情報などを照会",
    tools=tools,  
    system_message=(  
"""
あなたは「製品 & プロモーションエージェント（Product & Promotions Agent）」です。

- 構造化された情報源から、プロモーションのオファー、製品の在庫状況、
  適格条件、割引情報などを取得します。
- *ナレッジベース* のFAQ、利用規約、ベストプラクティスを活用して、
  回答を補足します。
- 事実に基づいた、最新の製品／プロモーション情報を提供します。
"""
    ),  
)  

data_analyst_agent = AssistantAgent(  
    name="DataAnalystAgent",  
    description="データに基づいて分析、計算、集計を行うためのエージェント。",
    model_client=model_client,  
    system_message=(
"""
あなたはデータアナリストです。
割り当てられたタスクに基づき、提供されたツールを使用してデータを分析し、結果を提供してください。
データを確認していない場合は、要求してください。
"""
    ),  
)  

In [ ]:
# 4. -----------------  Assemble Team -----------------  
participants: List[AssistantAgent] = [  
    crm_billing_agent,  
    product_promotions_agent,  
    analysis_planning_agent,      # orchestrator always concludes a cycle  
]  

## 停止条件
AutoGen には 無限ループを防止するため 8 つの組み込みの終了条件が定義されています。終了条件は以下のように OR 条件で指定できるのが便利です。



In [ ]:
# Define termination condition
max_msg_termination = MaxMessageTermination(max_messages=15)
text_termination = TextMentionTermination("TERMINATE")
time_terminarion = TimeoutTermination(120)
combined_termination = max_msg_termination | text_termination | time_terminarion


# SelectorGroupChat の実行

In [ ]:
selector_prompt = """Select an agent to perform task.

{roles}

Current conversation context:
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure the planner agent has assigned tasks before other agents start working.
Only select one agent.
"""

team_agent = SelectorGroupChat(  
    participants=participants,
    termination_condition=combined_termination,  
    selector_prompt=selector_prompt,
    model_client=model_client,
    allow_repeated_speaker=False,  # Allow an agent to speak multiple turns in a row.
)  

In [ ]:
with tracer.start_as_current_span("SelectorGroupChat") as rollspan:

    task = "ユーザーID:123 の出荷状況を確認してください。"

    await Console(team_agent.run_stream(task=task))

In [ ]:
with tracer.start_as_current_span("SelectorGroupChat") as rollspan:

    #task = "ユーザーID:123 の出荷状況を確認してください。あと頼んだのって何の商品だっけ"
    task = "商品ID:339の商品詳細を教えて"
    response = team_agent.run_stream(task=task)

    await Console(response)

In [ ]:
with tracer.start_as_current_span("SelectorGroupChat") as rollspan:

    #task = "ユーザーID:123 の出荷状況を確認してください。あと頼んだのって何の商品だっけ"
    task = "SNS分析を行い、日付ごとのツイート数を集計してください"
    response = team_agent.run_stream(task=task)

    await Console(response)

In [ ]:
with tracer.start_as_current_span("SelectorGroupChat") as rollspan:

    task = "2024年9月~10月の受注数を集計し、最も受注数が多いかった日付を教えて"
    response = team_agent.run_stream(task=task)

    await Console(response)

In [ ]:
with tracer.start_as_current_span("SelectorGroupChat") as rollspan:

    task = "2024年9月~10月の受注数を集計し、さらにSNS分析を行い、日付ごとのツイート数を集計した結果何かわかることはあるか？"
    response = team_agent.run_stream(task=task)

    await Console(response)